# Find the most similar job posting to our resume

## Objective 

Narrow down the set of job postings to those that are most similar to our resume in preparation for further analysis.
Workflow

## Workflow 

1. Obtain the resume from the GitHub repository. Transform job posting text and our resume into TF-IDF vectors using sklearn's TF-IDF vectorizer class.
2. Compute the cosine similarity between the vectorized resume and the job postings using sklearn's cosine similarity function.
3. Sort the job postings based on similarity to our resume, and choose an appropriate cutoff for selecting the most similar jobs. Store the most similar job postings in a new DataFrame for later use, and save the DataFrame to disk.